In [6]:
import math
from collections import Counter
from graphviz import Digraph
import pandas as pd
import numpy as np

In [7]:

# Node class for CART
class CARTTreeNode:
    def __init__(self, feature=None, threshold=None, children=None, label=None, value=None, is_leaf=False):
        self.feature = feature  # feature index for splitting
        self.threshold = threshold  # threshold value for splitting (for numeric features)
        self.children = children or {}  # dict of child nodes
        self.label = label  # class label for leaf nodes
        self.value = value  # class distribution at the node
        self.is_leaf = is_leaf  # explicitly track if this is a leaf node
        self.impurity = None  # gini impurity at this node

    def mark_leaf(self, label):
        self.is_leaf = True
        self.label = label

In [10]:
import math
from collections import Counter
from graphviz import Digraph
import pandas as pd
import numpy as np

# Fixed CART Node class
class CARTTreeNode:
    def __init__(self, feature=None, threshold=None, children=None, label=None, value=None, is_leaf=False, impurity=None):
        self.feature = feature  # feature index for splitting
        self.threshold = threshold  # threshold value for splitting (for numeric features)
        self.children = children or {}  # dict of child nodes
        self.label = label  # class label for leaf nodes
        self.value = value  # class distribution at the node
        self.is_leaf = is_leaf  # explicitly track if this is a leaf node
        self.impurity = impurity  # gini impurity at this node

    def mark_leaf(self, label):
        self.is_leaf = True
        self.label = label

# Fixed CART Decision Tree class
class CARTDecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None
        self.feature_names = None
        self.feature_types = None
    
    def gini_impurity(self, data):
        """Calculate Gini impurity for a dataset"""
        if len(data) == 0:
            return 0
            
        labels = [row[-1] for row in data]
        total = len(labels)
        if total == 0:
            return 0
            
        count = Counter(labels)
        return 1.0 - sum((c/total) ** 2 for c in count.values())
    
    def split_data(self, data, feature_idx, threshold, feature_type):
        """Split data based on feature and threshold"""
        if feature_type == 'numeric':
            left = [row for row in data if float(row[feature_idx]) <= threshold]
            right = [row for row in data if float(row[feature_idx]) > threshold]
            return left, right
        else:
            # For categorical features, split by equality
            left = [row for row in data if row[feature_idx] == threshold]
            right = [row for row in data if row[feature_idx] != threshold]
            return left, right
    
    def find_best_split(self, data, feature_indices):
        """Find the best split for CART using Gini impurity"""
        best_gini = float('inf')
        best_feature = None
        best_threshold = None
        best_split = None
        
        current_gini = self.gini_impurity(data)
        
        for feature_idx in feature_indices:
            feature_type = self.feature_types[feature_idx]
            feature_values = sorted(set(row[feature_idx] for row in data))
            
            if feature_type == 'numeric':
                # For numeric features, try thresholds between values
                for i in range(len(feature_values) - 1):
                    threshold = (float(feature_values[i]) + float(feature_values[i + 1])) / 2
                    left, right = self.split_data(data, feature_idx, threshold, feature_type)
                    
                    # Check minimum samples constraint
                    if len(left) < self.min_samples_leaf or len(right) < self.min_samples_leaf:
                        continue
                    
                    # Calculate weighted Gini impurity
                    left_gini = self.gini_impurity(left)
                    right_gini = self.gini_impurity(right)
                    weighted_gini = (len(left) * left_gini + len(right) * right_gini) / len(data)
                    
                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_feature = feature_idx
                        best_threshold = threshold
                        best_split = (left, right)
            else:
                # For categorical features, try each category as split
                for value in feature_values:
                    left, right = self.split_data(data, feature_idx, value, feature_type)
                    
                    # Check minimum samples constraint
                    if len(left) < self.min_samples_leaf or len(right) < self.min_samples_leaf:
                        continue
                    
                    # Calculate weighted Gini impurity
                    left_gini = self.gini_impurity(left)
                    right_gini = self.gini_impurity(right)
                    weighted_gini = (len(left) * left_gini + len(right) * right_gini) / len(data)
                    
                    if weighted_gini < best_gini:
                        best_gini = weighted_gini
                        best_feature = feature_idx
                        best_threshold = value
                        best_split = (left, right)
        
        # Only return if we found a meaningful split
        if best_gini < current_gini:
            return best_feature, best_threshold, best_split, best_gini
        else:
            return None, None, None, current_gini
    
    def build_tree(self, data, feature_indices, depth=0):
        """Recursively build the CART tree"""
        # Base cases
        if len(data) < self.min_samples_split:
            return self.create_leaf_node(data)
        
        if self.max_depth is not None and depth >= self.max_depth:
            return self.create_leaf_node(data)
        
        # Check if all labels are the same
        labels = [row[-1] for row in data]
        if len(set(labels)) == 1:
            return self.create_leaf_node(data)
        
        # Find best split
        best_feature, best_threshold, best_split, gini = self.find_best_split(data, feature_indices)
        
        # If no good split found, create leaf node
        if best_feature is None:
            return self.create_leaf_node(data)
        
        left_data, right_data = best_split
        
        # Create node with impurity parameter
        node = CARTTreeNode(
            feature=best_feature,
            threshold=best_threshold,
            impurity=gini
        )
        
        # For CART, we always do binary splits
        feature_type = self.feature_types[best_feature]
        
        if feature_type == 'numeric':
            node.children = {
                f" <= {best_threshold:.2f}": self.build_tree(left_data, feature_indices, depth + 1),
                f" > {best_threshold:.2f}": self.build_tree(right_data, feature_indices, depth + 1)
            }
        else:
            node.children = {
                f" = {best_threshold}": self.build_tree(left_data, feature_indices, depth + 1),
                f" ≠ {best_threshold}": self.build_tree(right_data, feature_indices, depth + 1)
            }
        
        return node
    
    def create_leaf_node(self, data):
        """Create a leaf node with the majority class"""
        labels = [row[-1] for row in data]
        if not labels:
            return CARTTreeNode(is_leaf=True, label=None)
        
        majority_label = Counter(labels).most_common(1)[0][0]
        return CARTTreeNode(is_leaf=True, label=majority_label)
    
    def fit(self, data, feature_names, feature_types):
        """Fit the CART tree to the data"""
        self.feature_names = feature_names
        self.feature_types = feature_types
        
        # All feature indices
        feature_indices = list(range(len(feature_names)))
        
        self.root = self.build_tree(data, feature_indices)
    
    def predict(self, sample):
        """Predict class for a single sample"""
        return self._predict_node(self.root, sample)
    
    def _predict_node(self, node, sample):
        """Recursively traverse tree for prediction"""
        if node.is_leaf:
            return node.label
        
        feature_idx = node.feature
        feature_type = self.feature_types[feature_idx]
        feature_value = sample[feature_idx]
        
        if feature_type == 'numeric':
            if float(feature_value) <= node.threshold:
                child_key = f" <= {node.threshold:.2f}"
            else:
                child_key = f" > {node.threshold:.2f}"
        else:
            if feature_value == node.threshold:
                child_key = f" = {node.threshold}"
            else:
                child_key = f" ≠ {node.threshold}"
        
        if child_key in node.children:
            return self._predict_node(node.children[child_key], sample)
        else:
            # If child not found, return the most common label from training
            return self._get_majority_class(node)
    
    def _get_majority_class(self, node):
        """Get majority class by traversing the tree (simplified)"""
        if node.is_leaf:
            return node.label
        
        # Get first available child and traverse
        for child in node.children.values():
            result = self._get_majority_class(child)
            if result is not None:
                return result
        return None
    
    def visualize(self, filename="cart_tree"):
        """Visualize the CART tree"""
        dot = Digraph()
        
        def add_nodes_edges(node, parent=None, edge_label=""):
            if node is None:
                return
                
            if node.is_leaf:
                label = f"Leaf\\nClass: {node.label}"
                color = "lightblue"
                shape = "ellipse"
            else:
                feature_name = self.feature_names[node.feature]
                if self.feature_types[node.feature] == 'numeric':
                    label = f"{feature_name}\\nGini: {node.impurity:.3f}"
                else:
                    label = f"{feature_name}\\nGini: {node.impurity:.3f}"
                color = "lightgrey"
                shape = "box"
            
            dot.node(str(id(node)), label=label, shape=shape, style='filled', fillcolor=color)
            
            if parent is not None:
                dot.edge(str(id(parent)), str(id(node)), label=str(edge_label))
            
            if not node.is_leaf:
                for edge_label, child in node.children.items():
                    add_nodes_edges(child, node, edge_label)
        
        add_nodes_edges(self.root)
        dot.render(filename, format="png", cleanup=True)
        print(f"CART Tree visualization saved as '{filename}.png'")
    
    def print_tree(self, node=None, depth=0):
        """Print tree structure for debugging"""
        if node is None:
            node = self.root
        if node is None:
            print("Tree is empty")
            return
            
        indent = "  " * depth
        if node.is_leaf:
            print(f"{indent}Leaf: Class = {node.label}")
        else:
            feature_name = self.feature_names[node.feature]
            impurity_str = f"{node.impurity:.3f}" if node.impurity is not None else "N/A"
            print(f"{indent}{feature_name} [Gini: {impurity_str}]")
            for edge_label, child in node.children.items():
                print(f"{indent}  -> {edge_label}:")
                self.print_tree(child, depth + 2)

In [11]:
if __name__ == "__main__":
    
    data = pd.read_csv("student.csv")
    
    # inspecting data
    print("Data columns:", data.columns.tolist())
    print("Data shape:", data.shape)
    print("First few rows:")
    print(data.head())
    
    # Use the correct column names based 
    attributes = ['Prior_Experience','Course','Time']
    
    # Define feature_types with INTEGER indices
    feature_types = {
        0: 'categorical',  # Prior_Experience (column 0 in data_list)
        1: 'categorical',  # Course (column 1 in data_list) 
        2: 'categorical',  # Time (column 2 in data_list)
    }

    # Convert to list and ensure we have the right columns
    data_list = data[attributes + [data.columns[-1]]].values.tolist()
    
    print(f"First data row: {data_list[0]}")
    print(f"Number of samples: {len(data_list)}")
    
    # Fit the CART decision tree
    cart_tree = CARTDecisionTree(max_depth=5, min_samples_split=2, min_samples_leaf=1)
    cart_tree.fit(data_list, attributes, feature_types)
    
    if cart_tree.root:
        if cart_tree.root.is_leaf:
            print("Root Feature: Leaf Node")
            print("Root Label:", cart_tree.root.label)
        else:
            print("Root Feature:", cart_tree.feature_names[cart_tree.root.feature])
            print("Root Threshold:", cart_tree.root.threshold)
            print("Root Gini Impurity:", f"{cart_tree.root.impurity:.3f}")
    
    # Tree structure for debugging
    print("\n=== CART Tree Structure ===")
    cart_tree.print_tree()
    
    # Visualize
    cart_tree.visualize("student_cart_tree")

    # Predict samples from the dataset
    print("\n=== Predictions on Sample Data ===")
    test_samples = [
        ['Yes', 'Programming', 'Night'],
        ['No', 'History', 'Day'],
        ['Yes', 'Mathematics', 'Day'],
        ['No', 'Programming', 'Day']
    ]
    
    for i, sample in enumerate(test_samples):
        prediction = cart_tree.predict(sample)
        print(f"Sample {i+1}: {sample} -> Prediction: {prediction}")
    
    # Test accuracy on training data
    print("\n=== Training Accuracy ===")
    correct = 0
    total = len(data_list)
    
    for i, row in enumerate(data_list):
        features = row[:-1]  # All except last column (target)
        actual = row[-1]     # Last column is target
        predicted = cart_tree.predict(features)
        
        if predicted == actual:
            correct += 1
    
    accuracy = correct / total
    print(f"Training Accuracy: {accuracy:.2f} ({correct}/{total} correct)")

Data columns: ['Student', 'Prior_Experience', 'Course', 'Time', 'Liked']
Data shape: (10, 5)
First few rows:
   Student Prior_Experience       Course   Time Liked
0        1              Yes  Programming    Day   Yes
1        2               No  Programming    Day    No
2        3              Yes      History  Night    No
3        4               No  Programming  Night   Yes
4        5              Yes      English    Day   Yes
First data row: ['Yes', 'Programming', 'Day', 'Yes']
Number of samples: 10
Root Feature: Course
Root Threshold: English
Root Gini Impurity: 0.444

=== CART Tree Structure ===
Course [Gini: 0.444]
  ->  = English:
    Leaf: Class = Yes
  ->  ≠ English:
    Course [Gini: 0.417]
      ->  = Mathematics:
        Leaf: Class = Yes
      ->  ≠ Mathematics:
        Course [Gini: 0.429]
          ->  = History:
            Leaf: Class = No
          ->  ≠ History:
            Time [Gini: 0.405]
              ->  = Day:
                Prior_Experience [Gini: 0.250]
   